# The goal of this notebook is to demonstrate the problems that I am having with the Tiler in MAAP OPS

In [10]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

import ipycmc
w = ipycmc.MapCMC()
w

MapCMC()

In [9]:
import requests
from pprint import pprint

titiler_endpoint_posting = "https://h9su0upami.execute-api.us-east-1.amazonaws.com"
titiler_endpoint_ops = "https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com"

In [3]:
# Collect mosaicdata
mosaicdata = {"mosaicjson": "0.0.2", "name": None, "description": None, "version": "1.0.0", "attribution": None, "minzoom": 10, "maxzoom": 18, "quadkey_zoom": 10, "bounds": [-101.00013888888888, 44.999861111111116, -99.9998611111111, 46.00013888888889], "center": [-100.5, 45.5, 10], "tiles": {"0213302200": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302201": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302210": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302211": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302202": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302203": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302212": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302213": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302220": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302221": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302230": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302231": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302222": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302223": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302232": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213302233": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213320000": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213320001": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213320010": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"], "0213320011": ["s3://maap-ops-workspace/graceal/N45W101.SRTMGL1.tif"]}}

In [4]:
r = requests.post(
    url=f"{titiler_endpoint_posting}/mosaicjson/mosaics",
    headers={
        "Content-Type": "application/vnd.titiler.mosaicjson+json",
    },
    #json=mosaicdata2.dict(exclude_none=True)).json()
    json=mosaicdata).json()
pprint(r)
tilejson_endpoint = list(filter(lambda x: x.get("rel") == "tilejson", dict(r)["links"]))[0].get('href')
print("Tile json endpoint Alex uses in his code: " + tilejson_endpoint)

xml_endpoint = list(filter(lambda x: x.get("rel") == "wmts", dict(r)["links"]))[0].get('href')
print("XML endpoint to try to pass directly to CMC " + xml_endpoint)

{'id': 'af200d53-620d-4381-b567-9f5a69b2739a',
 'links': [{'href': 'https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/af200d53-620d-4381-b567-9f5a69b2739a',
            'rel': 'self',
            'title': 'Self',
            'type': 'application/json'},
           {'href': 'https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/af200d53-620d-4381-b567-9f5a69b2739a/mosaicjson',
            'rel': 'mosaicjson',
            'title': 'MosiacJSON',
            'type': 'application/json'},
           {'href': 'https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/af200d53-620d-4381-b567-9f5a69b2739a/tilejson.json',
            'rel': 'tilejson',
            'title': 'TileJSON',
            'type': 'application/json'},
           {'href': 'https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/af200d53-620d-4381-b567-9f5a69b2739a/tiles/{z}/{x}/{y}',
            'rel': 'tiles',
            'title': 'Tiles',
     

## Attempt 1: Trying to pass XML endpoint directly into CMC

In [5]:
w.load_layer_config(xml_endpoint, "wmts/xml", {"handleAs": "wmts_raster"})

Outcome: A layer loads, but no tiles are shown on the map. After inspecting the network, I can tell that there is a 500 https error and you can see the error message at the following link [https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/c5c7ca12-1aa5-493e-9a6b-ae69f02412fe/tiles/10/224/364@1x.png?&TIME=2021-06-19](https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/mosaics/c5c7ca12-1aa5-493e-9a6b-ae69f02412fe/tiles/10/224/364@1x.png?&TIME=2021-06-19) The error is: {"detail":"'/vsis3/maap-ops-workspace/graceal/N45W101.SRTMGL1.tif' not recognized as a supported file format."}

## Attempt 2: Using tilejson_endpoint to pass to the Tiler and using that url for load_layer_config

In [12]:
# Create the request url by formatting using the Tiler ops endpoint
request_url = requests.get(
f"{titiler_endpoint_ops}/mosaicjson/WMTSCapabilities.xml?", params={"tile_format":"png", "tile_scale":"1", "pixel_selection":"first", "TileMatrixSetId":"WebMercatorQuad", "url": f"{tilejson_endpoint}", "resampling_method":"nearest", "return_mask":"true", "rescale":"0,1000"}
).url
request_url

'https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/mosaicjson/WMTSCapabilities.xml?tile_format=png&tile_scale=1&pixel_selection=first&TileMatrixSetId=WebMercatorQuad&url=https%3A%2F%2Fh9su0upami.execute-api.us-east-1.amazonaws.com%2Fmosaicjson%2Fmosaics%2Faf200d53-620d-4381-b567-9f5a69b2739a%2Ftilejson.json&resampling_method=nearest&return_mask=true&rescale=0%2C1000'

Outcome: No map layer loads and when you click the above link, you can see that there is the following error: {"detail":[{"loc":["query","url"],"msg":"field required","type":"value_error.missing"}]}

In [13]:
# Create the request url using the h9su endpoint
request_url2 = requests.get(
f"{titiler_endpoint_posting}/mosaicjson/WMTSCapabilities.xml?", params={"tile_format":"png", "tile_scale":"1", "pixel_selection":"first", "TileMatrixSetId":"WebMercatorQuad", "url": f"{tilejson_endpoint}", "resampling_method":"nearest", "return_mask":"true", "rescale":"0,1000"}
).url
request_url2

'https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/WMTSCapabilities.xml?tile_format=png&tile_scale=1&pixel_selection=first&TileMatrixSetId=WebMercatorQuad&url=https%3A%2F%2Fh9su0upami.execute-api.us-east-1.amazonaws.com%2Fmosaicjson%2Fmosaics%2Faf200d53-620d-4381-b567-9f5a69b2739a%2Ftilejson.json&resampling_method=nearest&return_mask=true&rescale=0%2C1000'

Outcome: No map layer loads and when you click the above link, you can see that there is the following error: {"detail":[{"loc":["query","url"],"msg":"field required","type":"value_error.missing"}]}

## Attempt 3: Creating a request URL by manually putting tilejson_endpoint into the Tiler

Putting tilejson_endpoint into Tiler ops /mosaicjson/WMTSCapabilities.xml
Outcome: Following error printed in Tiler: 
{
  "detail": "2 validation errors for Mosaic
  JSON\nmosaicjson\n  field required (type=value_error.missing)\ntiles\n  value is not a valid dict (type=type_error.dict)"
}

Putting tilejson_endpoint into h9su /mosaicjson/WMTSCapabilities.xml
Outcome: Following error printed in Tiler: 
{
  "detail": "2 validation errors for Mosaic
  JSON\nmosaicjson\n  field required (type=value_error.missing)\ntiles\n  value is not a valid dict (type=type_error.dict)"
}